In [1]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.device_count())

True
1


c:\Users\jimmy\.conda\envs\opencv-tracking-v5\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
